In [1]:
# As we use our own external modules, we need the folder src to be in the PYTHONPATH env variable.
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
from utils.config import FLEET_CAPACITY, PATH_SCENARIOS_REDUCED, PATH_INITIAL_ALLOCATION

In [3]:
scenarios = pd.read_pickle(PATH_SCENARIOS_REDUCED)

In [4]:
hex_ids = {*scenarios.index.get_level_values('start_hex_ids').unique()}
hex_ids = list(hex_ids.union({*scenarios.index.get_level_values('end_hex_ids').unique()}))

In [5]:
print(len(hex_ids))
print(FLEET_CAPACITY['kick_scooter'] / len(hex_ids))
print(FLEET_CAPACITY['kick_scooter'] % len(hex_ids))

17
11.764705882352942
13


In [6]:
initial_allocation = pd.DataFrame(index=pd.Index(hex_ids,name='hex_ids'))


In [7]:
for vehicle_type in list(scenarios.index.get_level_values('vehicle_types').unique()):
    allocation_per_hex = int(FLEET_CAPACITY[vehicle_type] / len(hex_ids))
    rest = FLEET_CAPACITY[vehicle_type] % len(hex_ids)

    initial_allocation[vehicle_type] = [allocation_per_hex] * len(hex_ids)

    increment_rest_selector = (initial_allocation.index[:rest], vehicle_type)

    initial_allocation.loc[increment_rest_selector] = initial_allocation.loc[increment_rest_selector] + 1

In [8]:
os.makedirs(os.path.dirname(PATH_INITIAL_ALLOCATION), exist_ok=True)
initial_allocation.to_pickle(PATH_INITIAL_ALLOCATION)